# Cargando dataset

## Importando librerías

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ruta = "/content/drive/MyDrive/Colab Notebooks/Big Data/supermercados_clean.csv"
data = pd.read_csv(ruta)
print(data.shape)

(2925720, 10)


# Precálculo y almacenamiento

In [ ]:
def precalcular_similitud(data, num_usuarios_similares = 10, num_minimo_visitas = 24):

  conteo_compras = data.groupby('CodCliente').size()

  #se hicieron 2 años de compras, queremos 24 visitas minimas que serían una por mes
  clientes_frecuentes = conteo_compras[conteo_compras >= num_minimo_visitas].index
  df_filtrado = data[data['CodCliente'].isin(clientes_frecuentes)]

  # Creamos un perfil de usuario y calcular la matriz de similitud
  perfil_usuario = pd.get_dummies(df_filtrado[['CodCliente', 'Categoria']], columns=['Categoria'])
  perfil_usuario = perfil_usuario.groupby('CodCliente').sum()

  # Calculamos la matriz de similitud entre todos los perfiles de usuario
  similitud = cosine_similarity(perfil_usuario)
  similitud_df = pd.DataFrame(similitud, index=perfil_usuario.index, columns=perfil_usuario.index)

  # Precalculamos los usuarios similares para cada cliente
  num_usuarios_similares = num_usuarios_similares
  usuarios_similares_precalculados = {}

  for cod_cliente in perfil_usuario.index:
      #ordenamos las puntuaciones de mayor a menor exceptuando el primero (mismo cliente) y convertimos a lista
      similares = similitud_df[cod_cliente].sort_values(ascending=False).index[1:num_usuarios_similares+1]
      usuarios_similares_precalculados[cod_cliente] = similares.tolist()

  # Guardamos los usuarios similares en un csv
  pd.DataFrame.from_dict(usuarios_similares_precalculados, orient='index').to_csv('usuarios_similares_precalculados.csv')
  return df_filtrado, perfil_usuario

# Carga y recomendación


In [ ]:
def categorias_mas_consumidas(cod_cliente, df_filtrado):
    compras_usuario = df_filtrado[df_filtrado['CodCliente'] == cod_cliente]
    frecuencia_categorias = compras_usuario.groupby('Categoria').size().sort_values(ascending=False)
    return frecuencia_categorias


In [ ]:
def cargar_usuarios_similares_y_recomendar(cod_cliente, df_filtrado, num_recomendaciones=5):
    # Cargamos la lista de usuarios similares precalculada
    usuarios_similares = pd.read_csv('usuarios_similares_precalculados.csv', index_col=0)
    lista_similares = usuarios_similares.loc[cod_cliente]

    # Inicializamos DataFrame para las recomendaciones y una lista para los usuarios similares
    productos_recomendados = pd.DataFrame()
    usuarios_que_influyen = []

    # Recopilamos productos comprados por usuarios similares
    for usuario in lista_similares:
        productos_usuario = df_filtrado[df_filtrado['CodCliente'] == usuario].copy()
        productos_usuario['Ganancia'] = abs(productos_usuario['PrecioUnitario'] - productos_usuario['CostoUnitario'])
        if not productos_usuario.empty:
            usuarios_que_influyen.append(usuario)

            productos_recomendados = pd.concat([productos_recomendados, productos_usuario], ignore_index=True)

    # Filtrar los productos ya comprados por el cliente
    productos_recomendados = productos_recomendados[~productos_recomendados['descripcion'].isin(df_filtrado[df_filtrado['CodCliente'] == cod_cliente]['descripcion'])]

    #ordenamos por ganancia (no se pudo) y luego por frecuencia para obtener las recomendaciones en base a estos 2

    recomendaciones_finales = productos_recomendados.groupby(['Categoria', 'descripcion']).agg({'Ganancia': 'mean', 'descripcion': 'size'}).rename(columns={'descripcion': 'Frecuencia'}).sort_values(by=['Frecuencia', 'Ganancia'], ascending=[False, False]).head(num_recomendaciones)
    #recomendaciones_finales = productos_recomendados.groupby(['Categoria', 'descripcion']).size().sort_values(ascending=False).head(num_recomendaciones)
    return recomendaciones_finales, usuarios_que_influyen


# Main

In [ ]:
df_filtrado, perfil_usuario = precalcular_similitud(data, 10, 24)

In [ ]:
cod_cliente = perfil_usuario.index[1]
categorias_cliente_principal = categorias_mas_consumidas(cod_cliente, df_filtrado)

print(f'El Cliente {cod_cliente} consume las siguientes categorias:\n{categorias_cliente_principal.index.tolist()[:5]}\n')
print(f'Recomendaciones para el cliente {cod_cliente}')

recomendaciones, usuarios_influyentes = cargar_usuarios_similares_y_recomendar(cod_cliente, df_filtrado)

print(recomendaciones)

# Creamos un DataFrame para los usuarios similares y sus categorías más consumidas
filas_usuarios = []
for usuario in usuarios_influyentes:
    categorias_usuario = categorias_mas_consumidas(usuario, df_filtrado)
    filas_usuarios.append({'Usuario': usuario, 'Categorías Principales': categorias_usuario.index.tolist()[:4]})

info_usuarios = pd.concat([pd.DataFrame([fila]) for fila in filas_usuarios], ignore_index=True)


print("\nUsuarios similares que influyeron en las recomendaciones: \n")
print(info_usuarios)

El Cliente 4 consume las siguientes categorias:
['JABONES', 'ORGANIZADORES', 'VERDURAS FRESCAS', 'VEGETALES ENVASADOS', 'GOLOSINAS']

Recomendaciones para el cliente 4
                                                  Ganancia  Frecuencia
Categoria        descripcion                                          
VERDURAS FRESCAS ZANAHORIA POR KILO               1.347400           9
LECHES LIQUIDAS  SV 3 LECHE PIL DESLACTOSADA UHT  3.442950           8
VERDURAS FRESCAS ARVEJA EXTRA PELADA UN           2.845375           8
                 PAPAS RICAPAPA ROJA GOURM        1.523388           8
                 ZANAHORIA EXTRA PICADA UN        1.426113           8

Usuarios similares que influyeron en las recomendaciones: 

   Usuario                             Categorías Principales
0  2445036  [ORGANIZADORES, JABONES, VERDURAS FRESCAS, BEB...
1  2710938  [GALLETAS, VERDURAS FRESCAS, SALSAS PREPARADAS...
2  2362020  [JABONES, CHOCOLATES, VERDURAS FRESCAS, GALLETAS]
3  6822202  [GALLETAS, GOL